In [ ]:
from ultralytics import YOLO

# Cargar el modelo preentrenado
model = YOLO('HeadBest.pt')


In [ ]:
import os
import shutil
import random

# Definir las rutas a los directorios
base_dir = '/path/to/dataSet'  # Reemplaza esto con la ruta correcta
unlabel_dir = os.path.join(base_dir, 'Unlabel')
train_images_dir = os.path.join(base_dir, 'images', 'train')
train_labels_dir = os.path.join(base_dir, 'labels', 'train')
processed_images_file = os.path.join(base_dir, 'processed_images.txt')

# Crear los directorios de destino si no existen
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)

def load_processed_images(processed_images_file):
    if os.path.exists(processed_images_file):
        with open(processed_images_file, 'r') as f:
            return set(line.strip() for line in f)
    return set()

def save_processed_images(processed_images_file, processed_images):
    with open(processed_images_file, 'w') as f:
        for image_path in processed_images:
            f.write(f"{image_path}\n")

def get_unlabeled_images(batch_size=100):
    processed_images = load_processed_images(processed_images_file)
    all_images = [os.path.join(unlabel_dir, f) for f in os.listdir(unlabel_dir) if f.endswith('.jpg') or f.endswith('.png')]

    # Filtrar las imágenes ya procesadas
    unprocessed_images = [img for img in all_images if img not in processed_images]

    # Seleccionar secuencialmente las primeras `batch_size` imágenes no procesadas
    return unprocessed_images[:batch_size]

# Ejemplo: Obtener un lote de 100 imágenes sin etiquetar
unlabeled_images = get_unlabeled_images(batch_size=100)



In [ ]:
def predict_and_label_images(images, model, confidence_threshold=0.7):
    labeled_images = []

    for image_path in images:
        results = model.predict(image_path, conf=confidence_threshold, verbose=False)

        for result in results:
            # Guardar solo si hay predicciones con confianza suficiente
            if result['conf'] >= confidence_threshold:
                labeled_images.append((image_path, result))

    return labeled_images

def save_predictions(predicted_labels, train_images_dir, train_labels_dir, processed_images):
    for image_path, predictions in predicted_labels:
        # Generar un nuevo nombre de archivo basado en los existentes en la carpeta de entrenamiento
        image_filename = os.path.basename(image_path)
        new_image_path = os.path.join(train_images_dir, image_filename)

        if os.path.exists(new_image_path):
            base_name, ext = os.path.splitext(image_filename)
            counter = 1
            new_image_path = os.path.join(train_images_dir, f"{base_name}_{counter}{ext}")
            while os.path.exists(new_image_path):
                counter += 1
                new_image_path = os.path.join(train_images_dir, f"{base_name}_{counter}{ext}")

        # Mover la imagen al directorio de entrenamiento
        shutil.move(image_path, new_image_path)

        # Guardar las etiquetas en el formato YOLO
        label_path = os.path.join(train_labels_dir, f"{os.path.splitext(image_filename)[0]}.txt")
        with open(label_path, 'w') as f:
            for prediction in predictions:
                class_id = prediction['class']
                x_center, y_center, width, height = prediction['bbox']
                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

        # Añadir la imagen al conjunto de imágenes procesadas
        processed_images.add(image_path)

def pseudolabeling_iteration(model, num_iterations=5, batch_size=100, confidence_threshold=0.7):
    processed_images = load_processed_images(processed_images_file)

    for i in range(num_iterations):
        print(f"Iteración {i+1}/{num_iterations}")

        # Seleccionar un lote de imágenes sin etiquetar
        unlabeled_images = get_unlabeled_images(batch_size)

        if not unlabeled_images:
            print("No quedan más imágenes sin etiquetar.")
            break

        # Obtener las predicciones para el lote de imágenes
        predicted_labels = predict_and_label_images(unlabeled_images, model, confidence_threshold)

        # Guardar las predicciones y mover las imágenes etiquetadas
        save_predictions(predicted_labels, train_images_dir, train_labels_dir, processed_images)

        # Guardar el estado de las imágenes procesadas
        save_processed_images(processed_images_file, processed_images)

# Ejecutar el proceso de pseudolabeling
pseudolabeling_iteration(model, num_iterations=5, batch_size=100, confidence_threshold=0.7)
